In [47]:
import stan
import pandas as pd
import nest_asyncio
import numpy as np

nest_asyncio.apply()

In [102]:
schools_code = """

data {
  int<lower=0> T;   // # time points (equally spaced)
  vector[T] y;      // mean corrected return at time t
  vector[T] X;
  
  vector[T] D_sun;
  vector[T] D_sat;
  vector[T] D_mon;
}

parameters {
  real mu;
  real psi;
  real d_sat;
  real d_sun;
  real d_mon;

  vector[T] epsilon_1;
  vector[T] epsilon_2;

  real<lower=0> eta_u;
  real<lower=0> eta_d;

  real kappa_h;
  real<lower=0> omega_h;
  real<lower=-1,upper=1> phi_h;
  real theta_h;

  vector<lower=0>[T] xi_d;
  vector<lower=0>[T] xi_u;

  // vector[3] alpha;
  array[T] simplex[3] q;

  vector[T] J_s;
  vector[T] h_s;
}
transformed parameters { 
  vector[T] J = J_s;
  vector[T] h = h_s;
  for (t in 1:T) {
    J[t] = -xi_d[t] * q[t][1] + 0 * q[t][2] + xi_u[t] * q[t][3];
  }

  for (t in 2:T) {
    h[t] = h[t-1] + kappa_h * (theta_h - h[t-1]) + phi_h * epsilon_1[t] + sqrt(omega_h) * epsilon_2[t];
  }
}
model {
  // Prior
  mu ~ normal(0, 10);
  psi ~ normal(0, 10);

  d_sat ~ normal(0, 10);
  d_sun ~ normal(0, 10);
  d_mon ~ normal(0, 10);

  kappa_h ~ normal(1, 6);

  omega_h ~ inv_gamma(3, 0.05);
  phi_h ~ normal(0, 0.5 * omega_h);
  theta_h ~ normal(0, 10);

  eta_u ~ inv_gamma(1.86, 0.43);
  eta_d ~ inv_gamma(1.86, 0.43);

  //alpha = rep_vector(1, 3);
  for (t in 1:T) {
    q[t] ~ dirichlet(rep_vector(1, 3));
  }

  // Likelihood
  for (t in 1:T) {
    xi_d[t] ~ exponential(eta_d);
    xi_u[t] ~ exponential(eta_u);
  }

  for (t in 1:T) {
    epsilon_1[t] ~ normal(0, 1);
    epsilon_2[t] ~ normal(0, 1);
  }
}
generated quantities {  
  vector[T] y_rep = y;
  for (t in 2:T) {
    y_rep[t] = y_rep[t-1] + mu + psi * X[t] + d_sat * D_sat[t] + d_sun * D_sun[t] + d_mon * D_mon[t] + epsilon_1[t] * sqrt(exp(h[t-1])) + J[t];
  }
}
"""

data = { 
    "T": 729,
    "y": pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/hour_4.csv')['log_diff'].to_numpy()[1:],
    "X": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/df_temp4.csv', usecols=['Temperature_C'])[1:-1].to_numpy()),

    "D_sun": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/sundays.csv').to_numpy()[:-1]),
    "D_sat": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/saturdays.csv').to_numpy()[:-1]),
    "D_mon": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/mondays.csv').to_numpy()[:-1])
    }

posterior = stan.build(schools_code, data=data)
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...

In file included from /Users/earl/Library/Caches/httpstan/4.11.0/models/ibmu4ixa/model_ibmu4ixa.cpp:2:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:4:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/Eigen.hpp:23:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/Eigen/Sparse:26:
In file included from /Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/Eigen/SparseCore:61:
/Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/include/Eigen/src/SparseCore/TriangularSolver.h:273

46 warnings generated.
ld: warning: duplicate -rpath '/Users/earl/anaconda3/envs/thesis_env/lib' ignored
ld: warning: object file (/Users/earl/anaconda3/envs/thesis_env/lib/python3.9/site-packages/httpstan/stan_services.o) was built for newer 'macOS' version (14.0) than being linked (11.0)

Building: 9.3s, done.Messages from stanc:
Warning in '/var/folders/w0/mvntm8ws53v7w48x57wdnh9w0000gn/T/httpstan_4kaibkra/model_ibmu4ixa.stan', line 62, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    calculation.
    calculation.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/

In [118]:
df.head()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,psi,d_sat,...,y_rep.720,y_rep.721,y_rep.722,y_rep.723,y_rep.724,y_rep.725,y_rep.726,y_rep.727,y_rep.728,y_rep.729
draws,,,,,,,,,,,,,,,,,,,,,
0,-6354.030279,0.980391,0.039888,6.0,64.0,1.0,9310.556177,-9.580994,-4.717051,19.903768,...,-23854.347373,-23870.03682,-23855.004534,-23860.346055,-23857.841359,-23858.169558,-23855.060536,-23846.407891,-23864.495644,-23855.581152
1,-6324.341984,0.965943,0.033171,5.0,57.0,1.0,9194.756049,1.267474,-6.765158,-14.860500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-6336.936023,0.919025,0.033995,10.0,1023.0,0.0,9276.050302,-14.649771,6.206807,6.573465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-6254.591458,0.999662,0.043100,10.0,1023.0,0.0,9193.885283,6.817163,-15.127983,14.928253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6329.057696,0.986239,0.039888,10.0,1023.0,0.0,9275.948185,4.097835,-8.151000,17.564583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
pd.options.display.max_seq_items = 10
df.columns

Index(['lp__', 'accept_stat__', 'stepsize__', 'treedepth__', 'n_leapfrog__',
       ...
       'y_rep.725', 'y_rep.726', 'y_rep.727', 'y_rep.728', 'y_rep.729'],
      dtype='object', name='parameters', length=8766)